In [1]:
#Q3
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the model architecture
def create_dense_nn(input_shape, num_classes, hidden_layers=2, neurons_per_layer=128, activation='relu'):
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=input_shape))  # Flatten the input images
    for _ in range(hidden_layers):
        model.add(layers.Dense(neurons_per_layer, activation=activation))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

# Example usage
input_shape = (28, 28)  # Assuming input images are 28x28 pixels
num_classes = 10  # Number of output classes
model = create_dense_nn(input_shape, num_classes)
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 10)                1290      
                                                                 
Total params: 118282 (462.04 KB)
Trainable params: 118282 (462.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [2]:
#Q4
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, SubsetRandomSampler
import numpy as np

# Define data transformations and download the SVHN dataset
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize images to 32x32
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Download the SVHN dataset
train_dataset = datasets.SVHN(root='./data', split='train', download=True, transform=transform)

# Create a subset of the dataset (25%)
subset_indices = np.random.choice(len(train_dataset), int(0.25 * len(train_dataset)), replace=False)
subset_sampler = SubsetRandomSampler(subset_indices)

# Create data loaders for training and validation
train_loader = DataLoader(train_dataset, batch_size=64, sampler=subset_sampler)
val_loader = DataLoader(train_dataset, batch_size=64, sampler=subset_sampler)

# Define LeNet-5 architecture with corrected pooling sizes
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # Flatten the output for fully connected layers
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define training and evaluation functions
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Calculate validation loss
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                outputs = model(images)
                val_loss += criterion(outputs, labels).item()

        print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_loader)}, Validation Loss: {val_loss/len(val_loader)}')

# Instantiate LeNet-5 model
model = LeNet5()

# Define loss function, optimizer, and train the model
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(model, train_loader, val_loader, criterion, optimizer)


100%|██████████| 182040794/182040794 [00:09<00:00, 19229897.61it/s]


Epoch 1/10, Training Loss: 1.6961113424251304, Validation Loss: 1.076909353093403
Epoch 2/10, Training Loss: 0.8316068453032796, Validation Loss: 0.6760756693235258
Epoch 3/10, Training Loss: 0.6169246307441166, Validation Loss: 0.533748140629991
Epoch 4/10, Training Loss: 0.5206320909880595, Validation Loss: 0.45585297712673295
Epoch 5/10, Training Loss: 0.46379317223818045, Validation Loss: 0.40897976786209733
Epoch 6/10, Training Loss: 0.4161881695433361, Validation Loss: 0.3530190512537956
Epoch 7/10, Training Loss: 0.37726627456394224, Validation Loss: 0.3402009753815388
Epoch 8/10, Training Loss: 0.3435512104100882, Validation Loss: 0.31827276020930617
Epoch 9/10, Training Loss: 0.31472860276699066, Validation Loss: 0.2831742942385142
Epoch 10/10, Training Loss: 0.2894549837874619, Validation Loss: 0.24076448772946302
